In [2]:
import os, time, torch, numpy as np
from torch.utils.tensorboard.writer import SummaryWriter
from src.TorchDSP.dataloader import MyDataset 
from src.TorchDSP.nneq import eqAMPBC
from src.TorchDSP.pbc import AmFoPBC, TorchSignal, TorchTime
from src.TorchDSP.loss import MSE, Qsq
from src.TorchSimulation.receiver import BER



window_size = 401
Nwindow = 100000
TBPL = 10000
train_data = MyDataset('dataset_A800/train.h5', Nch=[21], Rs=[80], Pch=[2], 
                       window_size=window_size, strides=1, Nwindow=Nwindow, truncate=20000, 
                       Tx_window=True, pre_transform='Rx_CDCDDLMS(taps=32,lr=[0.015625, 0.0078125])')

test_data = MyDataset('dataset_A800/test.h5', Nch=[21], Rs=[80], Pch=[2], 
                       window_size=window_size, strides=1, Nwindow=80000, truncate=20000, 
                       Tx_window=True, pre_transform='Rx_CDCDDLMS(taps=32,lr=[0.015625, 0.0078125])')

test_loader = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=True, drop_last=True)

print('Train Data number:',len(train_data))
print('Test Data number:',len(test_data))

def Test(net, dataloader):
    net.eval()
    mse, power, ber, N = 0,0,0, len(dataloader)
    with torch.no_grad():
        for Rx, Tx, info in dataloader:
            Rx, Tx, info = Rx.cuda(), Tx.cuda(), info.cuda()
            Rx = TorchSignal(val=Rx, t=TorchTime(0,0,1))
            y = net(Rx, info)
            symb = Tx[:,y.t.start:y.t.stop]
            loss = MSE(y.val, symb)
            mse += loss.item()
            power += MSE(symb, 0).item() 
            ber += np.mean(BER(y.val.view(-1, y.val.shape[-1]), symb.view(-1, symb.shape[-1]))['BER'])
    return {'MSE':mse/N, 'SNR': 10*np.log10(power/mse), 'BER':ber/N, 'Qsq': Qsq(ber/N)}

def write_log(writer, epoch, train_loss, metric):
    writer.add_scalar('Loss/train',  train_loss, epoch)
    writer.add_scalar('Loss/Test', metric['MSE'], epoch)
    writer.add_scalar('Metric/SNR', metric['SNR'], epoch)
    writer.add_scalar('Metric/BER', metric['BER'], epoch)
    writer.add_scalar('Metric/Qsq', metric['Qsq'], epoch)
    print(epoch, 'Train MSE:',  train_loss, 'Test MSE:', metric['MSE'],  'Qsq:', metric['Qsq'], flush=True)
    

Train Data number: 100000
Test Data number: 80000


## 0. AMFoPBC

In [2]:
os.makedirs('_outputs/log_test/AMFoPBC_Adam', exist_ok=True)
writer = SummaryWriter('_outputs/log_test/AMFoPBC_Adam')
epochs = 30

train_loader = torch.utils.data.DataLoader(train_data, batch_size=5000, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=5000, shuffle=True, drop_last=True)

net = AmFoPBC(L=window_size - 1, rho=1)
net.cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(epochs):
    train_loss = 0
    for i, (Rx, Tx, info) in enumerate(train_loader):
        Rx, Tx, info = Rx.cuda(), Tx.cuda(), info.cuda()
        Rx = TorchSignal(val=Rx, t=TorchTime(0,0,1))
        y = net(Rx, info)
        loss = MSE(y.val, Tx[:,y.t.start:y.t.stop])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()  
        writer.add_scalar('Loss/train_batch', loss.item(), epoch*len(train_loader)+i)
    scheduler.step()
    metric = Test(net, test_loader)
    write_log(writer, epoch, train_loss/len(train_loader), metric)


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 31.19 MiB is free. Including non-PyTorch memory, this process has 15.73 GiB memory in use. Of the allocated memory 14.70 GiB is allocated by PyTorch, and 693.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)